In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

## # Dataset with missing values

In [2]:
car_sales_missing = pd.read_csv("car-sales-extended-missing-data.csv")
car_sales_missing

,Make,Colour,Odometer (KM),Doors,Price
0,Honda,White,35431.0,4.0,15323.0
1,BMW,Blue,192714.0,5.0,19943.0
2,Honda,White,84714.0,4.0,28343.0
3,Toyota,White,154365.0,4.0,13434.0
4,Nissan,Blue,181577.0,3.0,14043.0
...,...,...,...,...,...
995,Toyota,Black,35820.0,4.0,32042.0
996,NaN,White,155144.0,3.0,5716.0
997,Nissan,Blue,66604.0,4.0,31570.0
998,Honda,White,215883.0,4.0,4001.0


#### 1. removing empty spaces that you have to remove in dataset

In [3]:
car_sales_missing.isna().sum()

Make             49
Colour           50
Odometer (KM)    50
Doors            50
Price            50
dtype: int64

In [4]:
len(car_sales_missing)

1000

In [5]:
# dealing with y-label empty spaces first (removing)
car_sales_missing.dropna(subset=["Price"], inplace = True)

In [6]:
car_sales_missing.isna().sum()

Make             47
Colour           46
Odometer (KM)    48
Doors            47
Price             0
dtype: int64

In [7]:
# samples reduced
len(car_sales_missing)

950

#### 2. splitting data into X and y labels

In [8]:
X = car_sales_missing.drop("Price", axis = 1)
y = car_sales_missing["Price"]

#### 3. filling the X features empty spaces

In [9]:
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

In [10]:
cat_imputer = SimpleImputer(strategy="constant", fill_value = "missing")
door_imputer = SimpleImputer(strategy = "constant", fill_value = 4)
num_imputer = SimpleImputer(strategy = "mean")

In [11]:
#defining columns
cat_features = ["Make", "Colour"]
door_feature = ["Doors"]
num_features = ["Odometer (KM)"]

In [12]:
# creating an imputer
imputer = ColumnTransformer([
            ("cat_imputer", cat_imputer, cat_features),
            ("num_imputer", num_imputer, num_features),
            ("door_imputer", door_imputer, door_feature)
])

In [13]:
# filling the empty spaces
filled_X = imputer.fit_transform(X)
filled_X

array([['Honda', 'White', 35431.0, 4.0],
       ['BMW', 'Blue', 192714.0, 5.0],
       ['Honda', 'White', 84714.0, 4.0],
       ...,
       ['Nissan', 'Blue', 66604.0, 4.0],
       ['Honda', 'White', 215883.0, 4.0],
       ['Toyota', 'Blue', 248360.0, 4.0]], dtype=object)

In [14]:
car_sales_filled = pd.DataFrame(filled_X, columns = ["Make", "Colour", "Odometer (KM)", "Doors"])
car_sales_filled

,Make,Colour,Odometer (KM),Doors
0,Honda,White,35431.0,4.0
1,BMW,Blue,192714.0,5.0
2,Honda,White,84714.0,4.0
3,Toyota,White,154365.0,4.0
4,Nissan,Blue,181577.0,3.0
...,...,...,...,...
945,Toyota,Black,35820.0,4.0
946,missing,White,155144.0,3.0
947,Nissan,Blue,66604.0,4.0
948,Honda,White,215883.0,4.0


In [15]:
car_sales_filled.isna().sum()

Make             0
Colour           0
Odometer (KM)    0
Doors            0
dtype: int64

#### 4. converting categorical columns into numerical columns

In [16]:
from sklearn.preprocessing import OneHotEncoder
#from sklearn.compose import ColumnTransformer

In [17]:
one_hot = OneHotEncoder()

In [18]:
# defining columns
categorical_features = ["Make", "Colour", "Doors"]

In [19]:
transformer = ColumnTransformer([("one_hot",  one_hot, categorical_features)],
                                remainder = "passthrough")

In [20]:
transformed_X = transformer.fit_transform(car_sales_filled)
transformed_X

<950x15 sparse matrix of type '<class 'numpy.float64'>'
	with 3800 stored elements in Compressed Sparse Row format>

In [21]:
pd.DataFrame(transformed_X)

,0
0,"(0, 1)\t1.0\n (0, 9)\t1.0\n (0, 12)\t1.0\n..."
1,"(0, 0)\t1.0\n (0, 6)\t1.0\n (0, 13)\t1.0\n..."
2,"(0, 1)\t1.0\n (0, 9)\t1.0\n (0, 12)\t1.0\n..."
3,"(0, 3)\t1.0\n (0, 9)\t1.0\n (0, 12)\t1.0\n..."
4,"(0, 2)\t1.0\n (0, 6)\t1.0\n (0, 11)\t1.0\n..."
...,...
945,"(0, 3)\t1.0\n (0, 5)\t1.0\n (0, 12)\t1.0\n..."
946,"(0, 4)\t1.0\n (0, 9)\t1.0\n (0, 11)\t1.0\n..."
947,"(0, 2)\t1.0\n (0, 6)\t1.0\n (0, 12)\t1.0\n..."
948,"(0, 1)\t1.0\n (0, 9)\t1.0\n (0, 12)\t1.0\n..."


### Fitting the model

In [22]:
from sklearn.model_selection import train_test_split

In [23]:
X_train, X_test, y_train, y_test = train_test_split(transformed_X, y, test_size = 0.2)

In [24]:
np.random.seed(42)
from sklearn.ensemble import RandomForestRegressor

In [25]:
model = RandomForestRegressor()
model.fit(X_train, y_train)

RandomForestRegressor()

### Evaluating the model

In [26]:
model.score(X_train, y_train)

0.8724665679867452